In [1]:
import mlflow

In [2]:
# Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-127-25-124.ap-south-1.compute.amazonaws.com:5000/")

In [4]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2025/06/20 20:59:23 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://youtube-insights-chrome-plugin-mlflow-bucket/547399241317235994', creation_time=1750433363378, experiment_id='547399241317235994', last_update_time=1750433363378, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [6]:
!pip install optuna


   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   ---------------------------------------- 2/2 [optuna]



In [8]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/150.0 MB 8.4 MB/s eta 0:00:18
    --------------------------------------- 3.1/150.0 MB 10.8 MB/s eta 0:00:14
   - -------------------------------------- 4.5/150.0 MB 8.6 MB/s eta 0:00:17
   - -------------------------------------- 6.0/150.0 MB 8.0 MB/s eta 0:00:18
   - -------------------------------------- 6.8/150.0 MB 7.4 MB/s eta 0:00:20
   -- ------------------------------------- 8.1/150.0 MB 7.1 MB/s eta 0:00:21
   -- ------------------------------------- 9.2/150.0 MB 6.8 MB/s eta 0:00:21
   -- ------------------------------------- 9.7/150.0 MB 6.2 MB/s eta 0:00:23
   -- ------------------------------------- 10.0/150.0 MB 5.8 MB/s eta 0:00:25
   -- ------------------------------------- 10.5/150.0 MB 5.3 MB/s eta 0:00:27
   -- ------------------------------------- 10.7/150.0 MB 5.1 MB/s eta 0:00:28
   --- ------------------------------------ 11.3/150.0 MB 4.8 MB/s 

In [9]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [10]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [11]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: Define ngram_range and max_features
ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 5: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

In [12]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [13]:
# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))

In [14]:
# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

In [15]:
# Run the experiment for XGBoost
run_optuna_experiment()

[I 2025-06-20 21:07:43,461] A new study created in memory with name: no-name-1ca7e3fb-d346-4db5-aadf-2ab7340f0259
[I 2025-06-20 21:08:18,944] Trial 0 finished with value: 0.6172098731760535 and parameters: {'n_estimators': 263, 'learning_rate': 0.010389856975126975, 'max_depth': 3}. Best is trial 0 with value: 0.6172098731760535.
[I 2025-06-20 21:08:45,494] Trial 1 finished with value: 0.5236601663711987 and parameters: {'n_estimators': 133, 'learning_rate': 0.0002139044343957055, 'max_depth': 4}. Best is trial 0 with value: 0.6172098731760535.
[I 2025-06-20 21:10:03,912] Trial 2 finished with value: 0.6780308195827083 and parameters: {'n_estimators': 79, 'learning_rate': 0.02847570936037148, 'max_depth': 9}. Best is trial 2 with value: 0.6780308195827083.
[I 2025-06-20 21:12:55,060] Trial 3 finished with value: 0.6127096686213009 and parameters: {'n_estimators': 133, 'learning_rate': 0.00015871120532059946, 'max_depth': 10}. Best is trial 2 with value: 0.6780308195827083.
[I 2025-06-2